In [83]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [84]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Part A

In [85]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self):
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [86]:
lang = Lang()
data = []
max_length = 0

i=0
with open("ferdousi.txt", encoding='utf8') as file:
    next(file)
    next(file)
    for line in file:
        try:
            B11 = line.strip()
            B12 = next(file).strip()
            B1 = B11+" "+B12

            B21 = next(file).strip()
            B22 = next(file).strip()
            B2 = B21+" "+B22

            data.append((B1, B2))
            
            lang.addSentence(B1)
            lang.addSentence(B2)
            max_length = (len(B11.split(' '))+len(B12.split(' '))) if (len(B11.split(' '))+len(B12.split(' '))) > max_length else max_length 
            max_length = (len(B21.split(' '))+len(B22.split(' '))) if (len(B21.split(' '))+len(B22.split(' '))) > max_length else max_length 

            i+=1
        except StopIteration:
            break
MAX_LENGTH = max_length+2

train_data = data[:int(len(data)*0.8)]
test_data = data[int(len(data)*0.8):]

In [87]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)

    def forward(self, input_, hidden, c):
        embedded = self.embedding(input_).view(1, 1, -1)
        output = embedded
        output, (hidden, c) = self.lstm(output, (hidden, c))
        return output, hidden, c

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
    
    def initC(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [88]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input_, hidden, c):
        output = self.embedding(input_).view(1, 1, -1)
        output = F.relu(output)
        output, (hidden, c) = self.lstm(output, (hidden, c))
        output = self.softmax(self.out(output[0]))
        return output, hidden, c

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
    
    def initC(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [89]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(lang, pair[0]).to(device)
    target_tensor = tensorFromSentence(lang, pair[1]).to(device)
    return (input_tensor, target_tensor)

In [167]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    encoder_hidden = encoder.initHidden() 
    encoder_c = encoder.initC()
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden, encoder_c = encoder(
            input_tensor[ei], encoder_hidden, encoder_c)

    decoder_input = torch.tensor([[SOS_token]], device=device)
    
    decoder_hidden = encoder_hidden
    decoder_c = encoder_c
    
    # Teacher forcing: Feed the target as the next input
    for di in range(target_length):
        decoder_output, decoder_hidden, decoder_c = decoder(
            decoder_input, decoder_hidden, decoder_c)
        loss += criterion(decoder_output, target_tensor[di])
        decoder_input = target_tensor[di]  # Teacher forcing

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [169]:
learning_rate = 0.005
encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [172]:
for i in range(1):
    losses = []
    print(len(data))
    for j, pair in enumerate(data):
        input_tensor, target_tensor = tensorsFromPair(pair)
        loss = train(input_tensor, target_tensor, encoder, decoder,
                    encoder_optimizer, decoder_optimizer, loss_fn)
        losses.append(loss)
        if j%10 ==1:
            print(j)
            print(f'average loss: {np.mean(losses)}')
            losses = []
    print(f'epoch: {i}')
    print(f'loss: {loss}')
    print()

24804
1
average loss: 3.778891118367513
11
average loss: 4.6399819102537645
21
average loss: 5.078195036588314
31
average loss: 6.875513392399836
41
average loss: 8.286774484035456
51
average loss: 7.861647128480854
61
average loss: 7.835332866990086
71
average loss: 7.775041256427288
81
average loss: 7.333172556228968
91
average loss: 7.7613386604113455
101
average loss: 7.563901643684244
111
average loss: 7.342155890049663
121
average loss: 7.287609167100058
131
average loss: 7.529071005584001
141
average loss: 7.552449583002141
151
average loss: 7.107726361772992
161
average loss: 7.322185722276551
171
average loss: 7.4667815247327285
181
average loss: 7.392352936912014
191
average loss: 7.508308018584775
201
average loss: 7.224179912402636
211
average loss: 7.589893621901375
221
average loss: 6.747961240572171
231
average loss: 7.256427667689569
241
average loss: 7.222695245752324
251
average loss: 7.090002226484961
261
average loss: 7.463775540539342
271
average loss: 7.3496953559

KeyboardInterrupt: 

Due to processing limitations, we were unable to fully train the network and then calculate losses and accuracy but we see that the loss is decreasing in the above outputs.

In [173]:
import matplotlib.pyplot as plt
# plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [206]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(lang, sentence)
        input_length = input_tensor.size(0)
        encoder_hidden = encoder.initHidden()
        encoder_c = encoder.initC()

        for ei in range(input_length):
            encoder_output, encoder_hidden, encoder_c = encoder(input_tensor[ei],
                                                     encoder_hidden, encoder_c)

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden
        decoder_c = encoder_c
        decoded_words = []
        max_length = 20

        for di in range(max_length):
            decoder_output, decoder_hidden, encoder_c = decoder(
                decoder_input, decoder_hidden, decoder_c)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()
        return ' '.join(decoded_words[:-1])

In [208]:
i=0
for pair in test_data:
    if i<10:
        input_sentence, target_sentence = pair
        output_sentence = evaluate(encoder, decoder, input_sentence)
        print(decodedWords)

        print()
        print(f'input sentence: ')
        print(input_sentence)
        print('model sentence: ')
        print(output_sentence)
        print('target sentence: ')
        print(target_sentence)
        print()
        i+=1

بسپارمش دیوش منهید زان آسا آسا آسا آسا آسا آسا آسا آسا آسا آسا آسا آسا آسا آسا آسا

input sentence: 
به آرایش چهره و فر و زیب نباید که گیرندت اندر فریب
model sentence: 
شارستان فوردین خودآگاه نویسید نجومیست همبر نویسید کرسی بخواندم سرانجمن چورفتند اودشمن تندباری بدجا پسندم دفتری برازد خارزن بتاج
target sentence: 
پس پردهٔ او بسی درخترست که با فر و بالا و با افسرست

بسپارمش دیوش منهید زان آسا آسا آسا آسا آسا آسا آسا آسا آسا آسا آسا آسا آسا آسا آسا

input sentence: 
پرستار زاده نیاید به کار اگر چند باشد پدر شهریار
model sentence: 
گبرش راآشکار مهرش گلان شگفتست بدکار جبرئیل سطالیس پسندم دفتری گازر نجست بدکار جبرئیل سطالیس پسندم دفتری گازر نجست
target sentence: 
نگر تا کدامست با شرم و داد به مادر که دارد ز خاتون نژاد

بسپارمش دیوش منهید زان آسا آسا آسا آسا آسا آسا آسا آسا آسا آسا آسا آسا آسا آسا آسا

input sentence: 
نبیره جهاندار فغفور چین ز پشت سپهدار خاقان چین
model sentence: 
زارست خویشتن برها توآییم انجا سرانجمن چورفتند گلان شگفتست پایکار گلان شگفتست پایکار گلان شگفتست پایکار گلان شگف

### Part B

In [ ]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self):
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
lang = Lang()
data = []
max_length = 0

i=0
with open("ferdousi.txt", encoding='utf8') as file:
    next(file)
    next(file)
    for line in file:
        try:
            B11 = line.strip()
            B12 = next(file).strip()
            B1 = B11+" "+B12

            B21 = next(file).strip()
            B22 = next(file).strip()
            B2 = B21+" "+B22

            data.append((B1, B2))
            
            lang.addSentence(B1)
            lang.addSentence(B2)
            max_length = (len(B11.split(' '))+len(B12.split(' '))) if (len(B11.split(' '))+len(B12.split(' '))) > max_length else max_length 
            max_length = (len(B21.split(' '))+len(B22.split(' '))) if (len(B21.split(' '))+len(B22.split(' '))) > max_length else max_length 

            i+=1
        except StopIteration:
            break
MAX_LENGTH = max_length+2

train_data = data[:int(len(data)*0.8)]
test_data = data[int(len(data)*0.8):]

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input_, hidden):
        embedded = self.embedding(input_).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
    
    def initC(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input_, hidden):
        output = self.embedding(input_).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
    
    def initC(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(lang, pair[0]).to(device)
    target_tensor = tensorFromSentence(lang, pair[1]).to(device)
    return (input_tensor, target_tensor)

In [ ]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    encoder_hidden = encoder.initHidden()     
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)

    decoder_input = torch.tensor([[SOS_token]], device=device)
    
    decoder_hidden = encoder_hidden
    
    # Teacher forcing: Feed the target as the next input
    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(
            decoder_input, decoder_hidden)
        loss += criterion(decoder_output, target_tensor[di])
        decoder_input = target_tensor[di]  # Teacher forcing

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
learning_rate = 0.005
encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
for i in range(1):
    losses = []
    print(len(data))
    for j, pair in enumerate(data):
        input_tensor, target_tensor = tensorsFromPair(pair)
        loss = train(input_tensor, target_tensor, encoder, decoder,
                    encoder_optimizer, decoder_optimizer, loss_fn)
        losses.append(loss)
        if j%10 ==1:
            print(j)
            print(f'average loss: {np.mean(losses)}')
            losses = []
    print(f'epoch: {i}')
    print(f'loss: {loss}')
    print()

Due to processing limitations, we were unable to fully train the network and then calculate losses and accuracy but it is very similar to the previous part.

In [ ]:
import matplotlib.pyplot as plt
# plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(lang, sentence)
        input_length = input_tensor.size(0)
        encoder_hidden = encoder.initHidden()
        encoder_c = encoder.initC()

        for ei in range(input_length):
            encoder_output, encoder_hidden, encoder_c = encoder(input_tensor[ei],
                                                     encoder_hidden, encoder_c)

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden
        decoder_c = encoder_c
        decoded_words = []
        max_length = 20

        for di in range(max_length):
            decoder_output, decoder_hidden, encoder_c = decoder(
                decoder_input, decoder_hidden, decoder_c)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()
        return ' '.join(decoded_words[:-1])

In [ ]:
i=0
for pair in test_data:
    if i<10:
        input_sentence, target_sentence = pair
        output_sentence = evaluate(encoder, decoder, input_sentence)
        print(decodedWords)

        print()
        print(f'input sentence: ')
        print(input_sentence)
        print('model sentence: ')
        print(output_sentence)
        print('target sentence: ')
        print(target_sentence)
        print()
        i+=1